In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier, plot_importance
import time
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import f1_score

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

# 실제로 적용해보기

## 데이터 프레임 다시 다듬기

In [2]:
original_df = pd.read_csv("dataframe(mile,meta).csv")

In [3]:
del original_df['participants']

In [4]:
original_df

,enrolled,hyhg,course_code,instructor,quota,max_mileage,mileage,major_quota,second_major,grade_quota_1,grade_quota_2,grade_quota_3,grade_quota_4,enrolled_courses,graduation,major,double,first_enroll,credits_rate,previous_credits_rate,grade_1,grade_2,grade_3,grade_4
0,1,20161,OMA1001-01-00,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,6,0,0,0,1,0.7925,0.9473,0,0,0,1
1,1,20161,OMA1001-01-00,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,6,0,0,0,1,0.7857,0.7142,0,0,0,1
2,1,20161,OMA1001-01-00,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,6,0,0,0,1,0.7428,0.8947,0,0,0,1
3,1,20161,OMA1001-01-00,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,5,0,0,0,1,0.7642,0.8421,0,0,0,1
4,1,20161,OMA1001-01-00,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,5,0,0,0,1,0.4337,0.4210,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985498,1,20192,KLI1006-01-00,김성숙,15.0,36.0,1,NaN,0.0,NaN,NaN,NaN,NaN,5,0,0,0,1,0.7500,0.9375,0,0,0,1
985499,1,20192,KLI1006-01-00,김성숙,15.0,36.0,1,NaN,0.0,NaN,NaN,NaN,NaN,5,0,0,0,1,0.2500,0.9375,0,1,0,0
985500,1,20192,KLI1006-01-00,김성숙,15.0,36.0,1,NaN,0.0,NaN,NaN,NaN,NaN,4,0,0,0,1,0.7500,0.9375,0,0,0,1
985501,1,20192,KLI1006-01-00,김성숙,15.0,36.0,1,NaN,0.0,NaN,NaN,NaN,NaN,4,0,0,0,1,0.7500,0.8750,0,0,0,1


In [5]:
original_df['course_code'] = original_df['course_code'].str.split('-',1).str[0]

In [6]:
original_df

,enrolled,hyhg,course_code,instructor,quota,max_mileage,mileage,major_quota,second_major,grade_quota_1,grade_quota_2,grade_quota_3,grade_quota_4,enrolled_courses,graduation,major,double,first_enroll,credits_rate,previous_credits_rate,grade_1,grade_2,grade_3,grade_4
0,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,6,0,0,0,1,0.7925,0.9473,0,0,0,1
1,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,6,0,0,0,1,0.7857,0.7142,0,0,0,1
2,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,6,0,0,0,1,0.7428,0.8947,0,0,0,1
3,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,5,0,0,0,1,0.7642,0.8421,0,0,0,1
4,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,5,0,0,0,1,0.4337,0.4210,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985498,1,20192,KLI1006,김성숙,15.0,36.0,1,NaN,0.0,NaN,NaN,NaN,NaN,5,0,0,0,1,0.7500,0.9375,0,0,0,1
985499,1,20192,KLI1006,김성숙,15.0,36.0,1,NaN,0.0,NaN,NaN,NaN,NaN,5,0,0,0,1,0.2500,0.9375,0,1,0,0
985500,1,20192,KLI1006,김성숙,15.0,36.0,1,NaN,0.0,NaN,NaN,NaN,NaN,4,0,0,0,1,0.7500,0.9375,0,0,0,1
985501,1,20192,KLI1006,김성숙,15.0,36.0,1,NaN,0.0,NaN,NaN,NaN,NaN,4,0,0,0,1,0.7500,0.8750,0,0,0,1


In [7]:
original_df.to_csv('dataframe_for_prediction.csv', index=False)

## 데이터프레임 확인

In [2]:
original_df = pd.read_csv('dataframe_for_prediction.csv')

In [3]:
original_df

,enrolled,hyhg,course_code,instructor,quota,max_mileage,mileage,major_quota,second_major,grade_quota_1,grade_quota_2,grade_quota_3,grade_quota_4,enrolled_courses,graduation,major,double,first_enroll,credits_rate,previous_credits_rate,grade_1,grade_2,grade_3,grade_4
0,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,6,0,0,0,1,0.7925,0.9473,0,0,0,1
1,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,6,0,0,0,1,0.7857,0.7142,0,0,0,1
2,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,6,0,0,0,1,0.7428,0.8947,0,0,0,1
3,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,5,0,0,0,1,0.7642,0.8421,0,0,0,1
4,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,5,0,0,0,1,0.4337,0.4210,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985498,1,20192,KLI1006,김성숙,15.0,36.0,1,NaN,0.0,NaN,NaN,NaN,NaN,5,0,0,0,1,0.7500,0.9375,0,0,0,1
985499,1,20192,KLI1006,김성숙,15.0,36.0,1,NaN,0.0,NaN,NaN,NaN,NaN,5,0,0,0,1,0.2500,0.9375,0,1,0,0
985500,1,20192,KLI1006,김성숙,15.0,36.0,1,NaN,0.0,NaN,NaN,NaN,NaN,4,0,0,0,1,0.7500,0.9375,0,0,0,1
985501,1,20192,KLI1006,김성숙,15.0,36.0,1,NaN,0.0,NaN,NaN,NaN,NaN,4,0,0,0,1,0.7500,0.8750,0,0,0,1


# 예측 시작 

In [21]:
from xgboost import XGBClassifier, plot_importance
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, f1_score, accuracy_score
from tqdm import tqdm

class CRpredict:
    def __init__(self, origin_df, course_code, instructor, quota, max_mileage,
                 mileage, major_quota, second_major, grade_quota_1, grade_quota_2, 
                 grade_quota_3, grade_quota_4, enrolled_courses, graduation, major, double, 
                 first_enroll, credits_rate, previous_credits_rate, grade):
        
        self.origin_df = origin_df
        self.course_code = str(course_code)
        self.instructor = str(instructor)
        self.quota = int(quota)
        self.max_mileage = int(max_mileage)
        self.mileage = int(mileage)
        self.major = int(major)
        self.double = int(double)
        self.major_quota = int(major_quota)
        self.second_major = int(second_major)
        self.enrolled_courses = int(enrolled_courses)
        self.graduation = int(graduation)
        self.first_enroll = int(first_enroll)
        self.credits_rate = float(credits_rate)
        self.previous_credits_rate = float(previous_credits_rate)
        self.grade = int(grade)
        self.grade_quota_1 = int(grade_quota_1)
        self.grade_quota_2 = int(grade_quota_2)
        self.grade_quota_3 = int(grade_quota_3)
        self.grade_quota_4 = int(grade_quota_4)

    def get_dataframe(self):
        temp = self.origin_df[self.origin_df['course_code']==self.course_code]
        new_df = temp[temp['instructor'] == self.instructor]
        return new_df
    
    def prediction(self):
        temp = self.origin_df[self.origin_df['course_code']==self.course_code]
        new_df = temp[temp['instructor'] == self.instructor]

        # Input 값으로 데이터프레임 만들어주기
        data = {'quota':[self.quota], 'max_mileage':[self.max_mileage], 'mileage' : [self.mileage], 
           'major_quota': [self.major_quota], 'second_major':[self.second_major], 
           'grade_quota_1':[self.grade_quota_1], 'grade_quota_2':[self.grade_quota_2],
           'grade_quota_3':[self.grade_quota_3], 'grade_quota_4':[self.grade_quota_4],
           'enrolled_courses':[self.enrolled_courses],'graduation':[self.graduation],
           'major':[self.major],'double':[self.double],'first_enroll':[self.first_enroll],
           'credits_rate':[self.credits_rate], 'previous_credits_rate':[self.previous_credits_rate],
            'grade_1':[0], 'grade_2':[0], 'grade_3':[0], 'grade_4':[0]}

        df_input = pd.DataFrame(data, columns = ['quota','max_mileage','mileage','major_quota',
                                  'second_major','grade_quota_1','grade_quota_2',
                                  'grade_quota_3','grade_quota_4','enrolled_courses',
                                  'graduation', 'major','double','first_enroll',
                                  'credits_rate','previous_credits_rate','grade_1','grade_2',
                                  'grade_3','grade_4'])
        # 학년 바꿔주기
        if self.grade == 1:
            df_input['grade_1'] = df_input['grade_1'].replace(0,1)
        elif self.grade == 2:
            df_input['grade_2'] = df_input['grade_2'].replace(0,1)
        elif self.grade == 3:
            df_input['grade_3'] = df_input['grade_3'].replace(0,1)
        elif self.grade == 4:
            df_input['grade_4'] = df_input['grade_4'].replace(0,1)
    
        # 학년별 정원 0인거 NaN값으로 바꿔주기
        if self.major_quota == 0:
            df_input['major_quota'] = df_input['major_quota'].replace(0, np.nan)
            
        df_input.loc[(df_input['grade_quota_1'] ==0) &
         (df_input['grade_quota_2'] ==0) &
         (df_input['grade_quota_3'] ==0) &
         (df_input['grade_quota_4'] ==0), ['grade_quota_1', 'grade_quota_2',
                                       'grade_quota_3', 'grade_quota_4']] = None
        self.dataframe = df_input
        
        X_train = new_df[list(new_df)[4:]]
        y_train = new_df['enrolled']
        X_test = df_input
    
        model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0,
              learning_rate=0.02, max_delta_step=0, max_depth=20,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, verbosity=1)
    
        # Fitting 하기
        start = time.time()
        model.fit(X_train, y_train)
        time_spent = time.time()-start

        # 예측하기
        prediction = model.predict(X_test)
        if prediction == 1:
            result = '가능!'
        elif prediction == 0:
            result = '불가능!ㅜ.ㅜ'
            
            
        # 마일리지 별로 수강신청 여부 예측하기
        optimum_mileage = self.max_mileage + 1
        passornot = []
        for i in range(self.max_mileage, 0, -1):
            optimum_mileage -= 1
            df_temp = df_input.copy()
            df_temp['mileage'] = df_temp['mileage'].replace(self.mileage,i)

            # 예측하기
            prediction_2 = model.predict(df_temp)
            passornot.append([prediction_2, optimum_mileage])
            self.optimum_mileage = optimum_mileage + 1
            self.passornot = passornot
            
            if prediction_2[0] == 0:
                break
            elif prediction_2[0] == 1:
                pass
        return result

## 예측을 하기 위한 데이터를 넣어주시고

In [138]:
predict_1st = CRpredict(original_df, 'YCS1002', '손의성', 38, 36, 36, 
                        0, 0, 0, 12, 13, 13,
                        6, 1, 0, 0, 1,
                        1, 0.8611, 1)

'''
parameter 순서
(self, origin_df, course_code, instructor, quota, max_mileage, mileage, 
major_quota, second_major, grade_quota_1, grade_quota_2, grade_quota_3, grade_quota_4,
enrolled_courses, graduation, major, double, first_enroll, 
credits_rate, previous_credits_rate,grade):
'''

'\nparameter 순서\n(self, origin_df, course_code, instructor, quota, max_mileage, mileage, \nmajor_quota, second_major, grade_quota_1, grade_quota_2, grade_quota_3, grade_quota_4,\nenrolled_courses, graduation, major, double, first_enroll, \ncredits_rate, previous_credits_rate,grade):\n'

## .prediction()을 하면 가능 여부를 알려주고

In [139]:
predict_1st.prediction()

'가능!'

## .optimum_mileage는 최적의 마일리지를 보여준다

In [140]:
predict_1st.optimum_mileage

36

In [137]:
predict_1st.passornot

[[array([1], dtype=int64), 36], [array([0], dtype=int64), 35]]

# 성능 계산하기

## 교수랑 과목 뽑기

In [4]:
original_df

,enrolled,hyhg,course_code,instructor,quota,max_mileage,mileage,major_quota,second_major,grade_quota_1,grade_quota_2,grade_quota_3,grade_quota_4,enrolled_courses,graduation,major,double,first_enroll,credits_rate,previous_credits_rate,grade_1,grade_2,grade_3,grade_4
0,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,6,0,0,0,1,0.7925,0.9473,0,0,0,1
1,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,6,0,0,0,1,0.7857,0.7142,0,0,0,1
2,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,6,0,0,0,1,0.7428,0.8947,0,0,0,1
3,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,5,0,0,0,1,0.7642,0.8421,0,0,0,1
4,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,5,0,0,0,1,0.4337,0.4210,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985498,1,20192,KLI1006,김성숙,15.0,36.0,1,NaN,0.0,NaN,NaN,NaN,NaN,5,0,0,0,1,0.7500,0.9375,0,0,0,1
985499,1,20192,KLI1006,김성숙,15.0,36.0,1,NaN,0.0,NaN,NaN,NaN,NaN,5,0,0,0,1,0.2500,0.9375,0,1,0,0
985500,1,20192,KLI1006,김성숙,15.0,36.0,1,NaN,0.0,NaN,NaN,NaN,NaN,4,0,0,0,1,0.7500,0.9375,0,0,0,1
985501,1,20192,KLI1006,김성숙,15.0,36.0,1,NaN,0.0,NaN,NaN,NaN,NaN,4,0,0,0,1,0.7500,0.8750,0,0,0,1


In [5]:
temp = original_df[['course_code', 'instructor']]

In [6]:
temp

,course_code,instructor
0,OMA1001,"김종철,홍기원"
1,OMA1001,"김종철,홍기원"
2,OMA1001,"김종철,홍기원"
3,OMA1001,"김종철,홍기원"
4,OMA1001,"김종철,홍기원"
...,...,...
985498,KLI1006,김성숙
985499,KLI1006,김성숙
985500,KLI1006,김성숙
985501,KLI1006,김성숙


In [7]:
temp = temp.drop_duplicates(subset=list(temp.columns))

In [8]:
temp

,course_code,instructor
0,OMA1001,"김종철,홍기원"
40,OMA1002,우환식
138,YCA1101,조재국
181,YCA1101,박정세
229,YCA1101,민경식
...,...,...
985265,MEU5480,정효일
985266,POL6104,이연호
985275,SLS6311,허진무
985276,SOC8872,김영미


In [9]:
data_sampling_1st = np.random.choice(9435, size=100, replace=False, p=None)

In [10]:
original_df.head()

,enrolled,hyhg,course_code,instructor,quota,max_mileage,mileage,major_quota,second_major,grade_quota_1,grade_quota_2,grade_quota_3,grade_quota_4,enrolled_courses,graduation,major,double,first_enroll,credits_rate,previous_credits_rate,grade_1,grade_2,grade_3,grade_4
0,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,6,0,0,0,1,0.7925,0.9473,0,0,0,1
1,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,6,0,0,0,1,0.7857,0.7142,0,0,0,1
2,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,6,0,0,0,1,0.7428,0.8947,0,0,0,1
3,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,5,0,0,0,1,0.7642,0.8421,0,0,0,1
4,1,20161,OMA1001,"김종철,홍기원",63.0,36.0,36,NaN,0.0,0.0,21.0,21.0,21.0,5,0,0,0,1,0.4337,0.4210,0,0,1,0


## 1st evalution
- n_estimators = 100

In [20]:
list_f1_score = []
for i in tqdm(data_sampling_1st):
    temp_course_code = original_df['course_code'].iloc[i]
    temp_instructor = original_df['instructor'].iloc[i]
    
    temp_df = original_df[original_df['course_code'] == temp_course_code]
    temp_df = original_df[original_df['instructor'] == temp_instructor]
    
    X = temp_df[list(temp_df)[4:]]
    y = temp_df['enrolled']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    
    model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0,
              learning_rate=0.02, max_delta_step=0, max_depth=20,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, verbosity=1)
    
    # Fitting 하기
    model.fit(X_train, y_train)

    # 예측하기
    y_pred = model.predict(X_test)
    test_score = f1_score(y_test, y_pred, average='weighted')
    list_f1_score.append(test_score)

 37%|█████████████████████████████▉                                                   | 37/100 [01:59<02:46,  2.64s/it]c:\programdata\miniconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:48<00:00,  1.98s/it]


In [21]:
summ = 0
for i in list_f1_score:
    summ += i
average = summ / len(list_f1_score)
average

0.9635471153751802

## 2nd evalution
- n_estimators = 300

In [23]:
data_sampling_2nd = np.random.choice(9435, size=100, replace=False, p=None)

In [24]:
list_f1_score_2nd = []
for i in tqdm(data_sampling_2nd):
    temp_course_code = original_df['course_code'].iloc[i]
    temp_instructor = original_df['instructor'].iloc[i]
    
    temp_df = original_df[original_df['course_code'] == temp_course_code]
    temp_df = original_df[original_df['instructor'] == temp_instructor]
    
    X = temp_df[list(temp_df)[4:]]
    y = temp_df['enrolled']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    
    model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0,
              learning_rate=0.02, max_delta_step=0, max_depth=20,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, verbosity=1)
    
    # Fitting 하기
    model.fit(X_train, y_train)

    # 예측하기
    y_pred = model.predict(X_test)
    test_score = f1_score(y_test, y_pred, average='weighted')
    list_f1_score_2nd.append(test_score)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [11:38<00:00,  4.54s/it]


In [25]:
summ = 0
for i in list_f1_score_2nd:
    summ += i
average = summ / len(list_f1_score)
average

0.968565266896629